In [81]:
pip install fake-useragent

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 161.7/161.7 kB 3.4 MB/s eta 0:00:00


In [58]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [82]:
import requests
from bs4 import BeautifulSoup
import re
import json
import pandas as pd
import time
import random
import asyncio
import aiohttp
import nest_asyncio
nest_asyncio.apply()
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor
from fake_useragent import UserAgent

import datetime
def date_time():return datetime.datetime.now(datetime.timezone(datetime.timedelta(hours=5, minutes=30))).strftime("%Y-%m-%d_%H-%M-%S")

- per page count show in website 15 but we can get 150 per page
-


- Inner page code

    params = {
        'backurl': 'H4sIAAAAAAAAAEXLMQ7CMAyF4dtkLQyMFmKBgQ5I9AJWYiBSEwfbKcrtKSKI7X+f9IaCd9LBcyqY41pKKP6xfwI6yhatTa2QwmEc+74aWtWTcC1fjtmzlKNwgt4Tg8MQhFT/7w5nai+WAE4NxWDj5piiwXbn9LZi+DwXzJ7CBTPNYFLJJQ4EP38De7CP8bMAAAA=',
    }

    response = requests.get(
        'https://app.companiesoffice.govt.nz/companies/app/ui/pages/companies/979583',
        params=params,
        headers=headers,
    )

In [60]:
headers = {
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',
    'Accept-Language': 'en-US,en;q=0.9',
    'Connection': 'keep-alive',
    'Referer': 'https://app.companiesoffice.govt.nz/companies/app/ui/pages/companies/search?mode=standard&type=entities&q=a&advancedPanel=true&entityTypes=ALL&entityStatusGroups=ALL',
    'Sec-Fetch-Dest': 'document',
    'Sec-Fetch-Mode': 'navigate',
    'Sec-Fetch-Site': 'same-origin',
    'Sec-Fetch-User': '?1',
    'Upgrade-Insecure-Requests': '1',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/137.0.0.0 Safari/537.36',
    'sec-ch-ua': '"Google Chrome";v="137", "Chromium";v="137", "Not/A)Brand";v="24"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"Windows"',
   }

params = {
    'q': 'a',
    'entityTypes': 'ALL',
    'entityStatusGroups': 'ALL',
    'incorpFrom': '',
    'incorpTo': '',
    'addressTypes': 'ALL',
    'addressKeyword': '',
    'start': '1',
    'limit': '1000',
    'sf': '',
    'sd': '',
    'advancedPanel': 'true',
    'mode': 'advanced',
}

# response = requests.get(
#     'https://app.companiesoffice.govt.nz/companies/app/ui/pages/companies/search',
#     params=params,
#     headers=headers,
# )

In [61]:
# response

In [62]:
# soup = BeautifulSoup(response.text, 'html.parser')

### Inner Pgae Details

In [63]:
cmpdf = pd.read_csv('/content/drive/MyDrive/Company Project/NZ_Company_Data_From_CompaniesOfficeGovtNz/data/nz_company_registry_scraper_companiesofficegovtnz_companies_list_16-06-2025.csv')

In [64]:
cmpdf.columns

Index(['Unnamed: 0', 'href', 'company_id', 'nzbn', 'company_name'], dtype='object')

In [65]:
cmpdf.head(1)

,Unnamed: 0,href,company_id,nzbn,company_name
0,0,javascript:viewCompanyDetails('174864');,174864,9429040149837,F F F ETCHELLS LIMITED


In [85]:
test_input_df = cmpdf[1000:2000]

In [86]:
ua = UserAgent()

# Get a random browser user-agent string
print(ua.random)

Mozilla/5.0 (Linux; Android 10; K) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/134.0.0.0 Mobile Safari/537.36


In [87]:
# Async HTML fetcher
async def inner_page_res(session, num):
  headers = {
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',
    'Accept-Language': 'en-US,en;q=0.9',
    'Connection': 'keep-alive',
    'Referer': 'https://app.companiesoffice.govt.nz/companies/app/ui/pages/companies/search?mode=standard&type=entities&q=a&advancedPanel=true&entityTypes=ALL&entityStatusGroups=ALL',
    'Sec-Fetch-Dest': 'document',
    'Sec-Fetch-Mode': 'navigate',
    'Sec-Fetch-Site': 'same-origin',
    'Sec-Fetch-User': '?1',
    'Upgrade-Insecure-Requests': '1',
    'User-Agent': ua.random,
    'sec-ch-ua': '"Google Chrome";v="137", "Chromium";v="137", "Not/A)Brand";v="24"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"Windows"',
   }

  async with semaphore:
    try:
        async with session.get(f'https://app.companiesoffice.govt.nz/companies/app/ui/pages/companies/{num}', headers=headers, timeout=10) as res:
            html = await res.text()
            return {'company_id': num, 'res': res}
    except Exception as e:
        return {'company_id': num, 'res': ''}

semaphore = asyncio.Semaphore(4)
# Async main
async def main():
    results = []
    async with aiohttp.ClientSession() as session:
        tasks = [inner_page_res(session, cid) for cid in test_input_df['company_id']]
        for f in tqdm(asyncio.as_completed(tasks), total=len(tasks), desc="Fetching company pages"):
            result = await f
            results.append(result)
    return results

# Entrypoint
if __name__ == "__main__":
    results = asyncio.run(main())

Fetching company pages: 100%|██████████| 1000/1000 [16:44<00:00,  1.00s/it]


In [88]:
def clean_text(el):
    return el.get_text(strip=True).replace('\xa0', ' ')

In [89]:
def clean_special(text):
    return re.sub(r'''[^a-zA-Z0-9\s,.'"]''', '', text)

In [97]:
def inner_page_extract(res_text, num):
    data = {}
    address = {}
    soup = BeautifulSoup(res_text, 'html.parser')
    try:
        for litag in soup.find('div', class_='readonly companySummary').find_all('div', class_='row'):
            try:

              if any(tag in str(litag).lower() for tag in ['registered', 'address']):
                    address[clean_text(litag.strong)] = clean_text(litag.br)
              else:
                    table_text = clean_text(litag.label)
                    data[table_text] = clean_special(clean_text(litag).split(table_text)[1])
            except:pass

        data['address'] = address
    except:pass
    data['company_id'] = num

    return data

In [98]:
result1 = []
for i in results:
  try:result1.append((i['company_id'], await i['res'].text()))
  except:pass

In [99]:
result1[0][1]

'\r\n\r\n\r\n<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Strict//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-strict.dtd">\r\n<html>\r\n  <head>\r\n    <meta http-equiv="Content-type" content="text/html;charset=UTF-8" />\r\n    \r\n    <meta http-equiv="Cache-Control" content="no-cache"/>\r\n    <meta http-equiv="Pragma" content="no-cache"/>\r\n    <meta name="robots" content="index, nofollow" />\r\n    \r\n    \r\n    <title>View All Details</title>\r\n\r\n    <meta http-equiv="x-ua-compatible" content="IE=edge"> \n<meta name="robots" content="noindex,nofollow"> \n<script src="//companies-register.companiesoffice.govt.nz/_resources/themes/base/dist/js/app.js?m=1750127355" type="text/javascript"></script> \n<link rel="stylesheet" href="//companies-register.companiesoffice.govt.nz/_resources/themes/base/dist/css/app.css?m=1750127355"> \n<link rel="stylesheet" href="//companies-register.companiesoffice.govt.nz/assets/_themes/theming-7e237c21e4b1b9682f42.css" media="all"> \n<link rel="sty

In [100]:
with ThreadPoolExecutor(max_workers=4) as exe:
    results2 = [i for i in tqdm(exe.map(inner_page_extract, [i[1] for i in result1], [i[0] for i in result1]), total=len(results))]

 74%|███████▍  | 744/1000 [00:36<00:12, 20.55it/s]


In [101]:
output = pd.json_normalize(results2)

In [102]:
output

,Company number:,NZBN:,Incorporation Date:,Company Status:,Entity type:,Constitution filed:,Ultimate holding company,Company record link:,company_id,address.Registered office address,address.Registered Office,address.Address for service,Directors,FRA Reporting Month:
0,5589,9429040961170,11 Feb 1948,Removed,NZ Limited Company,No,Not specified,httpsapp.companiesoffice.govt.nzco5589,5589,,"69 Rutherford Street, Lower Hutt\r\n, New Zealand",,NaN,NaN
1,886887,9429037948719,02 Dec 1997,Removed,NZ Limited Company,Yes,Not specified,httpsapp.companiesoffice.govt.nzco886887,886887,,"7 Simmonds Crescent, Rotorua\r\n, New Zealand","7 Simmonds Crescent, Rotorua\r\n, New Zealand",Showing 1 of 1 directors,NaN
2,62673,9429040678849,07 Mar 1962,Removed,NZ Limited Company,Yes,Not specified,httpsapp.companiesoffice.govt.nzco62673,62673,,"17 Hall St, Pukekohe\r\n, New Zealand","17 Hall St, Pukekohe\r\n, New Zealand",Showing 2 of 3 directors,NaN
3,49586,9429040728780,28 Apr 1952,Removed,NZ Limited Company,No,Not specified,httpsapp.companiesoffice.govt.nzco49586,49586,,"16 Elliott St, Papakura, Auckland\r\n, New Zea...",,NaN,NaN
4,1772708,9429034286852,14 Mar 2006,Removed,NZ Limited Company,Yes,Not specified,httpsapp.companiesoffice.govt.nzco1772708,1772708,,"Vogue House, First Floor, 9 Prince Avenue, Mou...","Vogue House, First Floor, 9 Prince Avenue, Mou...",Showing 2 of 3 directors,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
739,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1712136,NaN,NaN,NaN,NaN,NaN
740,1571696,9429035105121,01 Dec 2004,Removed,NZ Limited Company,Yes,No,httpsapp.companiesoffice.govt.nzco1571696,1571696,,"P K F Goldsmith Fox, Level 1, 100 Moorhouse Av...","P K F Goldsmith Fox, Level 1, 100 Moorhouse Av...",Showing 2 of 3 directors,NaN
741,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,186333,NaN,NaN,NaN,NaN,NaN
742,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5995067,NaN,NaN,NaN,NaN,NaN


In [103]:
output.to_csv('/content/drive/MyDrive/Company Project/NZ_Company_Data_From_CompaniesOfficeGovtNz/data/1k-2k_final_output_nz_company_registry_scraper_companiesofficegovtnz_companies.csv')